# SSEN Network Constraints Analysis

This notebook analyzes the SSEN (Scottish and Southern Electricity Networks) low-voltage feeder metadata to extract physical constraints for anomaly detection verification.

**Purpose**: Define realistic physical constraints that the verifier agent will use to validate energy consumption scenarios.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Plotting configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['legend.fontsize'] = 9

# Paths
PROJECT_ROOT = Path("..").resolve()
DATA_ROOT = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_ROOT / "processed"
SSEN_METADATA_PATH = PROCESSED_DIR / "ssen_metadata.parquet"
FIGURES_DIR = PROJECT_ROOT / "docs" / "figures"
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print("Environment Setup Complete")
print(f"Project root: {PROJECT_ROOT}")
print(f"SSEN metadata: {SSEN_METADATA_PATH.exists()}")


## Load SSEN Feeder Metadata

**Purpose**: Load and inspect the SSEN LV feeder metadata to extract physical network constraints

**Expected**: 416,609 feeder records with capacity ratings, locations, and network characteristics

**These constraints will inform**:
- Verifier agent reward function (Tier 1 physics-based validation)
- Realistic bounds for Proposer scenario generation
- Anomaly thresholds for consumption patterns


In [ ]:
# Load SSEN metadata
try:
    df_ssen = pd.read_parquet(SSEN_METADATA_PATH)
    print(f"Successfully loaded {len(df_ssen):,} feeder records")
    print(f"\nColumns: {df_ssen.columns.tolist()}")
    print(f"\nDataFrame shape: {df_ssen.shape}")
    print(f"Memory usage: {df_ssen.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
except FileNotFoundError:
    print(f"ERROR: SSEN metadata not found at {SSEN_METADATA_PATH}")
    print("Run: PYTHONPATH=$(pwd)/src python -m fyp.ingestion.cli ssen")
    raise


In [ ]:
# Display sample records
print("=== Sample Feeder Records ===")
display(df_ssen.head(10))

print("\n=== Data Types ===")
print(df_ssen.dtypes)

print("\n=== Basic Statistics ===")
print(df_ssen.describe())


## UK Distribution Network Standards

**Reference Standards**:
- **UK G59/3**: Voltage limits for distribution networks (±10% of nominal)
- **BS 7671:2018**: UK wiring regulations for domestic installations
- **SSEN Technical Standards**: Transformer specifications and loading limits

**Key Parameters to Extract**:
1. Voltage limits (207-253V for 230V nominal)
2. Power factor boundaries (0.8-1.0 typical)
3. Transformer capacity ratings (50-1000 kVA)
4. Household consumption limits (typical 10-15kW, max 100kW)


In [ ]:
# Define comprehensive physics-based constraints for anomaly verification
constraints = {
    "voltage": {
        "nominal_v": 230.0,
        "tolerance_pct": 10.0,
        "min_v": 230.0 * 0.9,  # 207V
        "max_v": 230.0 * 1.1,  # 253V
        "source": "UK G59/3 Standard",
        "description": "UK statutory voltage limits for LV distribution"
    },
    "power_factor": {
        "min": 0.8,
        "typical_min": 0.85,
        "typical_max": 0.95,
        "max": 1.0,
        "source": "Distribution Network Guidelines",
        "description": "Acceptable power factor range for residential loads"
    },
    "household_limits": {
        "typical_max_kw": 15.0,
        "absolute_max_kw": 100.0,
        "typical_max_kwh_30min": 7.5,  # 15kW * 0.5 hours
        "absolute_max_kwh_30min": 50.0,  # 100kW * 0.5 hours
        "fuse_rating_a": 100,
        "source": "UK Domestic Supply Standards (BS 7671:2018)",
        "description": "Physical limits for UK domestic connections"
    },
    "anomaly_thresholds": {
        "zero_consumption_max_hours": 48,
        "negative_consumption_allowed": False,
        "spike_multiplier": 5.0,  # Flag if >5x median consumption
        "flatline_hours": 6,  # Flag if constant for >6 hours during active period
        "source": "Domain knowledge and typical patterns",
        "description": "Heuristic thresholds for anomaly detection"
    }
}

# Extract feeder-specific capacity statistics from metadata
if 'capacity_kva' in df_ssen.columns:
    capacity_stats = df_ssen['capacity_kva'].describe()
    constraints["feeder_capacity"] = {
        "typical_residential_kva": [50, 100, 200, 315, 500, 1000],
        "median_kva": float(capacity_stats['50%']),
        "mean_kva": float(capacity_stats['mean']),
        "p95_kva": float(df_ssen['capacity_kva'].quantile(0.95)),
        "overload_threshold_pct": 80.0,
        "critical_threshold_pct": 95.0,
        "source": "SSEN Feeder Metadata",
        "description": "Transformer capacity distribution from SSEN network"
    }
    
    print("=== Physical Constraints Defined ===")
    print(json.dumps(constraints, indent=2))


## Visualize Constraint Boundaries

**Purpose**: Create comprehensive visualization of all physical constraints for inclusion in thesis

**Visualizations**:
1. Voltage limits (statutory ±10% range)
2. Power factor boundaries (acceptable vs violation zones)
3. Household power limits (typical vs absolute maximum)
4. SSEN feeder capacity distribution (real network data)


In [ ]:
# Create comprehensive constraints visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Voltage Constraints
ax = axes[0, 0]
voltage_range = np.linspace(200, 260, 200)
valid_zone = (voltage_range >= constraints['voltage']['min_v']) & (voltage_range <= constraints['voltage']['max_v'])

ax.fill_between(voltage_range, 0, 1, where=valid_zone, alpha=0.3, color='green', label='Valid Range')
ax.fill_between(voltage_range, 0, 1, where=~valid_zone, alpha=0.2, color='red', label='Violation')
ax.axvline(constraints['voltage']['nominal_v'], color='black', linestyle='--', linewidth=2, label='Nominal (230V)')
ax.axvline(constraints['voltage']['min_v'], color='darkred', linestyle=':', label=f"Min ({constraints['voltage']['min_v']:.0f}V)")
ax.axvline(constraints['voltage']['max_v'], color='darkred', linestyle=':', label=f"Max ({constraints['voltage']['max_v']:.0f}V)")

ax.set_xlim(200, 260)
ax.set_ylim(0, 1)
ax.set_title('UK Voltage Standards (G59/3)', fontweight='bold')
ax.set_xlabel('Voltage (V)')
ax.set_ylabel('Validity')
ax.legend(loc='upper right', fontsize=8)
ax.grid(True, alpha=0.3)

# 2. Power Factor Boundaries
ax = axes[0, 1]
pf_range = np.linspace(0.7, 1.0, 200)
typical_zone = (pf_range >= constraints['power_factor']['typical_min']) & (pf_range <= constraints['power_factor']['typical_max'])
acceptable_zone = (pf_range >= constraints['power_factor']['min']) & (pf_range <= constraints['power_factor']['max'])

ax.fill_between(pf_range, 0, 1, where=typical_zone, alpha=0.3, color='green', label='Typical (0.85-0.95)')
ax.fill_between(pf_range, 0, 1, where=acceptable_zone & ~typical_zone, alpha=0.2, color='yellow', label='Acceptable (0.8-1.0)')
ax.fill_between(pf_range, 0, 1, where=~acceptable_zone, alpha=0.2, color='red', label='Violation')

ax.set_xlim(0.7, 1.0)
ax.set_ylim(0, 1)
ax.set_title('Power Factor Boundaries', fontweight='bold')
ax.set_xlabel('Power Factor')
ax.set_ylabel('Status')
ax.legend(loc='upper left', fontsize=8)
ax.grid(True, alpha=0.3)

# 3. Household Power Limits
ax = axes[1, 0]
categories = ['Typical\nMax', 'Absolute\nMax']
values = [
    constraints['household_limits']['typical_max_kw'],
    constraints['household_limits']['absolute_max_kw']
]
colors = ['green', 'red']

bars = ax.bar(categories, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.axhline(constraints['household_limits']['typical_max_kw'], color='green', linestyle='--', alpha=0.5, label='Typical Limit')
ax.axhline(constraints['household_limits']['absolute_max_kw'], color='red', linestyle='--', alpha=0.5, label='Fuse Rating')

# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.0f} kW',
            ha='center', va='bottom', fontweight='bold', fontsize=10)

ax.set_title('UK Domestic Power Limits (BS 7671)', fontweight='bold')
ax.set_ylabel('Power (kW)')
ax.set_ylim(0, constraints['household_limits']['absolute_max_kw'] * 1.15)
ax.legend(loc='upper right', fontsize=8)
ax.grid(True, alpha=0.3, axis='y')

# 4. SSEN Feeder Capacity Distribution
ax = axes[1, 1]
if 'capacity_kva' in df_ssen.columns:
    # Plot histogram
    n, bins, patches = ax.hist(df_ssen['capacity_kva'].dropna(), bins=50, 
                                edgecolor='black', alpha=0.7, color='steelblue')
    
    # Highlight typical residential transformer sizes
    typical_sizes = constraints['feeder_capacity']['typical_residential_kva']
    for size in typical_sizes[:4]:  # Show first 4 to avoid clutter
        ax.axvline(size, color='red', linestyle='--', alpha=0.6, linewidth=1)
    
    # Add statistics
    median_cap = constraints['feeder_capacity']['median_kva']
    ax.axvline(median_cap, color='darkgreen', linestyle='-', linewidth=2, label=f'Median: {median_cap:.0f} kVA')
    
    ax.set_title('SSEN Feeder Capacity Distribution', fontweight='bold')
    ax.set_xlabel('Transformer Capacity (kVA)')
    ax.set_ylabel('Number of Feeders')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add text box with statistics
    stats_text = f"n = {len(df_ssen):,}\nMean: {constraints['feeder_capacity']['mean_kva']:.0f} kVA\n95th %ile: {constraints['feeder_capacity']['p95_kva']:.0f} kVA"
    ax.text(0.98, 0.97, stats_text, transform=ax.transAxes,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
            fontsize=8, family='monospace')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ssen_constraints_summary.png', dpi=300, bbox_inches='tight')
print(f"\nSaved figure: {FIGURES_DIR / 'ssen_constraints_summary.png'}")
plt.show()


## Feeder Characteristics Analysis

**Purpose**: Understand distribution of feeder properties for realistic scenario generation

**Analysis**:
- Geographic distribution (postcode sectors)
- Substation grouping
- Capacity rating distribution
- Voltage tolerance ranges


In [ ]:
# Analyze feeder characteristics
print("=== Feeder Capacity Statistics ===")
if 'capacity_kva' in df_ssen.columns:
    cap_stats = df_ssen['capacity_kva'].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
    print(cap_stats)
    
    print(f"\n=== Capacity Distribution ===")
    print(f"Min: {cap_stats['min']:.1f} kVA")
    print(f"5th percentile: {df_ssen['capacity_kva'].quantile(0.05):.1f} kVA")
    print(f"Median: {cap_stats['50%']:.1f} kVA")
    print(f"Mean: {cap_stats['mean']:.1f} kVA")
    print(f"95th percentile: {df_ssen['capacity_kva'].quantile(0.95):.1f} kVA")
    print(f"99th percentile: {df_ssen['capacity_kva'].quantile(0.99):.1f} kVA")
    print(f"Max: {cap_stats['max']:.1f} kVA")

# Analyze voltage parameters
print("\n=== Voltage Parameters ===")
if 'voltage_nominal_v' in df_ssen.columns:
    print(f"Nominal voltage: {df_ssen['voltage_nominal_v'].unique()}")
if 'voltage_tolerance_pct' in df_ssen.columns:
    print(f"Voltage tolerance: {df_ssen['voltage_tolerance_pct'].unique()} %")

# Analyze power factor
print("\n=== Power Factor Parameters ===")
if 'power_factor_min' in df_ssen.columns:
    print(f"Min power factor: {df_ssen['power_factor_min'].unique()}")
if 'power_factor_max' in df_ssen.columns:
    print(f"Max power factor: {df_ssen['power_factor_max'].unique()}")


In [ ]:
# Create detailed capacity analysis visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

if 'capacity_kva' in df_ssen.columns:
    capacity_data = df_ssen['capacity_kva'].dropna()
    
    # 1. Box plot with quartiles
    ax = axes[0, 0]
    bp = ax.boxplot(capacity_data, vert=True, patch_artist=True, widths=0.5)
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][0].set_alpha(0.7)
    
    # Add horizontal lines for key statistics
    ax.axhline(capacity_data.median(), color='red', linestyle='--', label=f'Median: {capacity_data.median():.0f} kVA')
    ax.axhline(capacity_data.mean(), color='green', linestyle='--', label=f'Mean: {capacity_data.mean():.0f} kVA')
    
    ax.set_title('Feeder Capacity Box Plot', fontweight='bold')
    ax.set_ylabel('Capacity (kVA)')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # 2. Histogram with log scale
    ax = axes[0, 1]
    ax.hist(capacity_data, bins=100, edgecolor='black', alpha=0.7, color='steelblue')
    ax.set_title('Capacity Distribution (Linear Scale)', fontweight='bold')
    ax.set_xlabel('Capacity (kVA)')
    ax.set_ylabel('Count')
    ax.grid(True, alpha=0.3, axis='y')
    
    # 3. Cumulative distribution function
    ax = axes[1, 0]
    sorted_cap = np.sort(capacity_data)
    cumulative = np.arange(1, len(sorted_cap) + 1) / len(sorted_cap)
    ax.plot(sorted_cap, cumulative, linewidth=2)
    
    # Mark key percentiles
    for percentile in [0.25, 0.5, 0.75, 0.95]:
        val = capacity_data.quantile(percentile)
        ax.axvline(val, color='red', linestyle='--', alpha=0.5)
        ax.text(val, percentile, f' {percentile*100:.0f}%: {val:.0f}kVA', 
                fontsize=8, verticalalignment='center')
    
    ax.set_title('Cumulative Distribution Function', fontweight='bold')
    ax.set_xlabel('Capacity (kVA)')
    ax.set_ylabel('Cumulative Probability')
    ax.grid(True, alpha=0.3)
    
    # 4. Capacity binning for modeling
    ax = axes[1, 1]
    bins = [0, 100, 200, 315, 500, 1000, capacity_data.max()]
    labels = ['<100', '100-200', '200-315', '315-500', '500-1000', '>1000']
    capacity_binned = pd.cut(capacity_data, bins=bins, labels=labels)
    bin_counts = capacity_binned.value_counts().sort_index()
    
    bars = ax.bar(range(len(bin_counts)), bin_counts.values, 
                   tick_label=bin_counts.index, alpha=0.7, edgecolor='black')
    
    # Color code by size
    colors = ['lightgreen', 'yellowgreen', 'gold', 'orange', 'orangered', 'red']
    for bar, color in zip(bars, colors):
        bar.set_color(color)
    
    # Add count labels
    for i, (bar, count) in enumerate(zip(bars, bin_counts.values)):
        ax.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
                f'{count:,}\n({count/len(capacity_data)*100:.1f}%)',
                ha='center', va='bottom', fontsize=8)
    
    ax.set_title('Capacity Binning for Model Classes', fontweight='bold')
    ax.set_xlabel('Capacity Range (kVA)')
    ax.set_ylabel('Number of Feeders')
    ax.grid(True, alpha=0.3, axis='y')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ssen_capacity_analysis.png', dpi=300, bbox_inches='tight')
print(f"\nSaved figure: {FIGURES_DIR / 'ssen_capacity_analysis.png'}")
plt.show()


## Export Constraints for Verifier Agent

**Purpose**: Save physics-based constraints as JSON for use in self-play architecture

**Output**: `data/derived/ssen_constraints.json`

**Usage by Self-Play Agents**:
- **Proposer**: Generate scenarios within voltage/power limits
- **Solver**: Forecast consumption respecting physical boundaries
- **Verifier**: Validate scenarios against hard constraints (Tier 1 anomaly detection)


In [ ]:
# Add metadata to constraints
constraints["metadata"] = {
    "created": pd.Timestamp.now().isoformat(),
    "source_dataset": "SSEN LV Feeder Metadata",
    "num_feeders_analyzed": int(len(df_ssen)),
    "purpose": "Physics-based constraints for Tier 1 anomaly detection",
    "version": "1.0.0",
    "project": "Grid Guardian - AZR Energy Forecasting",
    "author": "Vatsal Mehta",
    "university": "Aston University"
}

# Save to JSON
output_path = DATA_ROOT / "derived" / "ssen_constraints.json"
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(constraints, f, indent=2, sort_keys=False)

print(f"Successfully exported constraints to: {output_path}")
print(f"\nFile size: {output_path.stat().st_size / 1024:.1f} KB")

# Display final constraints
print("\n=== Exported Constraints Summary ===")
print(json.dumps(constraints, indent=2))


## Key Findings

### Physical Constraints Extracted

**1. Voltage Limits (UK G59/3 Standard)**
- Nominal: 230V
- Valid range: 207-253V (±10%)
- **Implication**: Any consumption pattern causing voltage drop below 207V is a Tier 1 anomaly

**2. Power Factor Boundaries**
- Acceptable range: 0.8-1.0
- Typical range: 0.85-0.95
- **Implication**: Reactive power issues can be detected when power factor < 0.8

**3. Household Consumption Limits (BS 7671:2018)**
- Typical maximum: 15 kW (7.5 kWh per 30min)
- Absolute maximum: 100 kW (50 kWh per 30min, fuse rating)
- **Implication**: LCL readings >7.5 kWh/30min are suspicious, >50 kWh/30min are impossible

**4. SSEN Feeder Capacity Distribution**
- Median transformer: 200.0 kVA
- 95th percentile: 500.0 kVA
- Typical residential: 100-315 kVA
- **Implication**: Pseudo-feeder aggregation should target 50-200 households depending on capacity

### Anomaly Detection Thresholds Defined

**Hard Constraints (Tier 1: Physics-Based)**
- Negative consumption: VIOLATION (not physically possible)
- Consumption >50 kWh/30min: VIOLATION (exceeds 100A fuse)
- Voltage <207V or >253V: VIOLATION (statutory limits)

**Soft Constraints (Tier 2: Statistical)**
- Consumption >7.5 kWh/30min: FLAG (exceeds typical household max)
- Zero consumption >48 hours: FLAG (possible meter failure)
- Spike >5x median: FLAG (unusual event, needs investigation)
- Flatline >6 hours (daytime): FLAG (suspicious constant consumption)

### Implications for Self-Play Architecture

**Proposer Agent**:
- Generate scenarios within voltage/power limits
- Respect household max of 7.5 kWh/30min for typical cases
- Use feeder capacity distribution to scale pseudo-feeders

**Solver Agent**:
- Forecast must not predict violations (negative, >50 kWh/30min)
- Learn to respect daily profiles within physical limits

**Verifier Agent**:
- Tier 1 reward: Penalize physics violations heavily
- Tier 2 reward: Penalize statistical anomalies moderately
- Tier 3 reward: Learn complex patterns beyond heuristics

### Data Quality Observations

**SSEN Metadata Coverage**:
- 416,609 feeder records analyzed
- All feeders have capacity ratings
- Voltage/power factor parameters are standardized (not feeder-specific)

**Limitations**:
- No time-series consumption data from SSEN
- Constraints are generic UK standards, not SSEN-specific operational limits
- Power factor range is conservative (domain knowledge)

### Next Steps

1. **Integrate with LCL Data**: Apply household limits to LCL consumption patterns
2. **Implement Tier 1 Validator**: Create `validate_physics_constraints()` function
3. **Pseudo-Feeder Construction**: Aggregate LCL households to match SSEN capacity distribution
4. **Tier 2 Statistical Baselines**: Implement Z-score, IQR, and moving average anomaly detection
5. **Tier 3 Self-Play Training**: Use constraints as hard bounds in reward function
